In [1]:
## 모듈로딩
from Korpora import Korpora                     # Open Korean Dataset
from konlpy.tag import *                        # 형태소 분석기
import spacy                                    # 형태소 분석기
# from torchtext.data.utils import get_tokenizer  # Pytorch 자연어처리 모듈
# from torchtext.vocab import *                   # Pytorch 자연어처리 단어사전 관련 모듈
from torch.utils.data import Dataset, DataLoader# Pytorch Dataset 관련 모듈
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import re

In [2]:
nsmc=Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\kjy19\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\kjy

In [3]:
nsmcDF=pd.DataFrame(nsmc.test)
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [4]:
class TextDataset(Dataset):
    def __init__(self,feature,label):
        self.feature=feature
        self.label=label
        self.length=feature.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.feature.iloc[index],self.label.iloc[index]

In [6]:
nsmcDS=TextDataset(nsmcDF['text'],nsmcDF['label'])

In [7]:
n=0
for feature,label in nsmcDS:
    print(feature,label)
    n+=1
    if n==10:
        break

굳 ㅋ 1
GDNTOPCLASSINTHECLUB 0
뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아 0
지루하지는 않은데 완전 막장임... 돈주고 보기에는.... 0
3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠?? 0
음악이 주가 된, 최고의 음악영화 1
진정한 쓰레기 0
마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다 0
갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한cg남무 아 그립다 동사서독같은 영화가 이건 3류아류작이다 0
이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네.. 1


In [8]:
okt=Okt()

In [9]:
def generateToken(dataset):
    tokenlists=[]
    for text,label in dataset:
        # 한글빼고 다지우기
        text=re.sub('[^ㄱ-ㅎ가-힣]+',' ',text)
        tokenlist=''
        # 기본형으로 토큰 주세요
        tokens=okt.pos(text,norm=True,stem=True)
        for token in tokens:
            if (len(token[0])>1):
                tokenlist=tokenlist+' '+str(token[0])
        tokenlists.append(tokenlist)
    return tokenlists

def vectorize(tokenlist,padding=20):
    ### 단어 사전 생성 및 변환
    cv=CountVectorizer()
    cv.fit_transform(tokenlist)
    vo=cv.vocabulary_
    vectorlist=tokenlist.copy()
    for idx,sen in enumerate(vectorlist):
        senlist=sen.split(' ')[1:]
        length=len(senlist)
        if length<padding:
            for ind,st in enumerate(senlist):
                senlist[ind]=vo[st]
            vectorlist[idx]=senlist+([0]*(padding-length))
        else:
            for ind,st in enumerate(senlist):
                senlist[ind]=vo[st]
            vectorlist[idx]=senlist[:padding]

    return vectorlist

In [10]:
tokenlist=generateToken(nsmcDS)
vectorlist=vectorize(tokenlist,padding=20)